In [1]:
import requests
import pandas as pd
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_project_data(project_id):
    API_URL = f"https://pad.crc.nd.edu/api/v2/projects/{project_id}/cards"
    try:
        # fetch_data_from_api
        r = requests.get(url=API_URL,verify=False)  # NOTE: Using verify=False due to a SSL issue, I need a valid certificate, then I will remove this parameter.
        r.raise_for_status() # Raise an exception if the status is not 200
        data = r.json()
        df = pd.json_normalize(data)
        return df
    except requests.exceptions.RequestException as e:
        print(e)
        print(f"Error accessing project data {project_id}: {r.status_code}")
        return None


In [2]:
project_id = 13
df = get_project_data(project_id)

In [3]:
df.columns.to_list()

['id',
 'sample_name',
 'test_name',
 'user_name',
 'date_of_creation',
 'raw_file_location',
 'processed_file_location',
 'processing_date',
 'camera_type_1',
 'notes',
 'sample_id',
 'quantity',
 'deleted',
 'issue',
 'project.id',
 'project.user_name',
 'project.project_name',
 'project.annotation',
 'project.test_name',
 'project.sample_names.sample_names',
 'project.neutral_filler',
 'project.qpc20',
 'project.qpc50',
 'project.qpc80',
 'project.qpc100',
 'project.notes']

In [4]:
## Counting unique values to identify categories in the 'sample_name' column

# Normalize case to lowercase and replace spaces with dashes in 'sample_name'
df['sample_name'] = df['sample_name'].str.lower().str.replace(' ', '-', regex=False)

# Counting unique values to identify categories
print(df['sample_name'].value_counts())

amoxicillin-rerun                    149
blank-pad-(no-reagents)              110
acetaminophen                         44
blank-pad-(stamped-with-reagents)     11
albendazole                            8
amoxicillin                            6
quinine                                3
                                       2
Name: sample_name, dtype: int64


In [23]:
# Sample random 3 cases for each unique sample_name
# If a group has less than 3 entries, it takes all from that group
sample_cases_per_sample_name = df.groupby('sample_name').apply(lambda x: x.sample(min(len(x), 10)))

print("\nRandom sample cases for unique quantities:")
sample_cases_per_sample_name[['id', 'sample_name']]


Random sample cases for unique quantities:


id  \
sample_name                                    
                                  326  34728   
                                  325  31169   
acetaminophen                     7    13685   
                                  21   13710   
                                  4    13682   
                                  43   13747   
                                  20   13709   
                                  10   13688   
                                  2    13680   
                                  23   13712   
                                  22   13711   
                                  41   13745   
albendazole                       327  47292   
                                  328  47293   
                                  323  21857   
                                  324  21858   
                                  332  47602   
                                  331  47600   
                                  329  47294   
                                  330  47578   
amoxicillin                       317  14160   
                                  320  14164   
                                  318  14162   
                                  321  14165   
                                  322  14166   
                                  319  14163   
amoxicillin-rerun                 82   13888   
                                  114  13920   
                                  87   13893   
                                  55   13861   
                                  69   13875   
                                  120  13926   
                                  50   13856   
                                  51   13857   
                                  136  13942   
                                  141  13947   
blank-pad-(no-reagents)           240  14053   
                                  284  14097   
                                  243  14056   
                                  228  14041   
                                  230  14043   
                                  215  14028   
                                  302  14115   
                                  198  14011   
                                  244  14057   
                                  269  14082   
blank-pad-(stamped-with-reagents) 313  14126   
                                  316  14129   
                                  314  14127   
                                  308  14121   
                                  307  14120   
                                  311  14124   
                                  312  14125   
                                  315  14128   
                                  306  14119   
                                  309  14122   
quinine                           40   13744   
                                  38   13742   
                                  39   13743   

                                                             sample_name  
sample_name                                                               
                                  326                                     
                                  325                                     
acetaminophen                     7                        acetaminophen  
                                  21                       acetaminophen  
                                  4                        acetaminophen  
                                  43                       acetaminophen  
                                  20                       acetaminophen  
                                  10                       acetaminophen  
                                  2                        acetaminophen  
                                  23                       acetaminophen  
                                  22                       acetaminophen  
                                  41                       acetaminophen  
albendazole                       327                        al

In [24]:
import ipyplot
import matplotlib.pyplot as plt



label_column = 'sample_name'
data = sample_cases_per_sample_name.copy()
# Define the base URL
base_url = "https://pad.crc.nd.edu"

# Concatenate to get the full image URLs
data['url'] = base_url + data['processed_file_location']

# The 'url' column now contains the full URLs to the images
images = data['url'].values
labels = data[label_column].values

# Labels for the tabs - unique quantities
labels_list_filtered = data[label_column].unique().tolist()

# Show with tabs
ipyplot.plot_class_tabs(
    images, labels, tabs_order=labels_list_filtered, img_width=150
)

plt.show() 

In [26]:
# Count unique 'sample_id's and total 'id's for each 'sample_name' and 'quantity' group
counts_per_group = df[df['sample_name'].isin(['blank-pad-(stamped-with-reagents)', 'blank-pad-(no-reagents)'])].groupby(['sample_name', 'quantity']).agg({
    'sample_id': 'nunique',  # Counts unique sample_ids
    'id': 'count'            # Counts total ids
})

print("Counts of 'sample_id's and 'id's by 'sample_name' and 'quantity' group:")
counts_per_group

Counts of 'sample_id's and 'id's by 'sample_name' and 'quantity' group:


,,sample_id,id
sample_name,quantity,,
blank-pad-(no-reagents),100,109,110
blank-pad-(stamped-with-reagents),100,11,11


In [18]:
sample_cases_per_sample_name

id  \
sample_name                                    
                                  325  31169   
                                  326  34728   
acetaminophen                     42   13746   
                                  0    13678   
                                  15   13693   
albendazole                       331  47600   
                                  329  47294   
                                  327  47292   
amoxicillin                       319  14163   
                                  318  14162   
                                  322  14166   
amoxicillin-rerun                 111  13917   
                                  130  13936   
                                  104  13910   
blank-pad-(no-reagents)           262  14075   
                                  254  14067   
                                  212  14025   
blank-pad-(stamped-with-reagents) 308  14121   
                                  307  14120   
                                  314  14127   
quinine                           38   13742   
                                  40   13744   
                                  39   13743   

                                                             sample_name  \
sample_name                                                                
                                  325                                      
                                  326                                      
acetaminophen                     42                       acetaminophen   
                                  0                        acetaminophen   
                                  15                       acetaminophen   
albendazole                       331                        albendazole   
                                  329                        albendazole   
                                  327                        albendazole   
amoxicillin                       319                        amoxicillin   
                                  318                        amoxicillin   
                                  322                        amoxicillin   
amoxicillin-rerun                 111                  amoxicillin-rerun   
                                  130                  amoxicillin-rerun   
                                  104                  amoxicillin-rerun   
blank-pad-(no-reagents)           262            blank-pad-(no-reagents)   
                                  254            blank-pad-(no-reagents)   
                                  212            blank-pad-(no-reagents)   
blank-pad-(stamped-with-reagents) 308  blank-pad-(stamped-with-reagents)   
                                  307  blank-pad-(stamped-with-reagents)   
                                  314  blank-pad-(stamped-with-reagents)   
quinine                           38                             quinine   
                                  40                             quinine   
                                  39                             quinine   

                                                test_name  \
sample_name                                                 
                                  325  12LanePADKenya2015   
                                  326  12LanePADKenya2015   
acetaminophen                     42   12LanePADKenya2015   
                                  0    12LanePADKenya2015   
                                  15   12LanePADKenya2015   
albendazole                       331  12LanePADKenya2015   
                                  329  12LanePADKenya2015   
                                  327  12LanePADKenya2015   
amoxicillin                       319  12LanePADKenya2015   
                                  318  12LanePADKenya2015   
                                  322  12LanePADKenya2015   
amoxicillin-rerun                 111  12LanePADKenya2015   
                                  130  12LanePADKenya2015   
                         

In [ ]:
print(data['url'].to_list())

In [ ]:
!source ../.venv/bin/activate

In [ ]:
!which pip

In [ ]:
import ipyplot